In [1]:
%load_ext autoreload
%autoreload 2

DATA_FOLDER = '/home/azuredwp08/data/haystack/'

import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 150)

In [2]:
df = pd.read_csv(DATA_FOLDER + 'scenario1/haystack_device.csv')
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M:%S')
df['ft_count'] = df['file_tree'].apply(lambda r: len(r.split(';')) if pd.notnull(r) else 0)
device_df = df

In [23]:
df = pd.read_csv(DATA_FOLDER + 'scenario1/haystack_email.csv', nrows=30000)
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M:%S')
email_df = df

In [42]:
df = pd.read_csv(DATA_FOLDER + 'scenario1/haystack_psychometric.csv', nrows=10000)
df['user'] = df['user_id']
# df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M:%S')
psycho_df = df
df.head()

,employee_name,user_id,O,C,E,A,N,user
0,Henry Benedict Mcleod,HBM0676,44,30,11,18,19,HBM0676
1,Aidan Bevis Hayden,ABH0349,32,41,38,18,34,ABH0349
2,Karina Rhoda Cruz,KRC0842,25,18,38,10,20,KRC0842
3,Ian Lucian Mcdowell,ILM1441,37,31,17,24,32,ILM1441
4,Eagan Jarrod Vega,EJV0094,39,38,24,41,33,EJV0094


In [45]:
psycho_df[psycho_df['employee_name'].str.contains('Derrick')]

,employee_name,user_id,O,C,E,A,N,user


### Extract domains

In [24]:
email_df['to'] = email_df['to'].apply(lambda v: v.split(';'))

In [25]:
import re
re.match(r'.+@(.+)', 'ferohajnovic@gmail.com').group(1)

'gmail.com'

In [34]:
email_df['domain_from'] = email_df['from'].apply(lambda e: re.match(r'.+@(.+)', e).group(1))

In [36]:
email_df['domain_from'].value_counts()

dtaa.com               19345
juno.com                 843
verizon.net              789
gmail.com                780
optonline.net            778
sbcglobal.net            738
bellsouth.net            738
yahoo.com                737
hotmail.com              710
comcast.net              702
charter.net              668
aol.com                  661
netzero.com              641
cox.net                  640
earthlink.net            609
msn.com                  578
boeing.com                10
lockheedmartin.com         9
raytheon.com               9
northropgrumman.com        6
harris.com                 5
hp.com                     4
Name: domain_from, dtype: int64

In [37]:
email_df['domain_to'] = email_df['to'].apply(lambda es: [re.match(r'.+@(.+)', e).group(1) for e in es])

In [38]:
email_df['domain_to']

0                                     [dtaa.com, dtaa.com]
1                                               [dtaa.com]
2                                               [dtaa.com]
3                                               [dtaa.com]
4                                               [dtaa.com]
5                                               [dtaa.com]
6                             [bellsouth.net, netzero.com]
7        [dtaa.com, dtaa.com, dtaa.com, dtaa.com, dtaa....
8                                          [optonline.net]
9                                       [hp.com, dtaa.com]
10                [dtaa.com, dtaa.com, dtaa.com, dtaa.com]
11                                              [dtaa.com]
12                                              [dtaa.com]
13                                              [dtaa.com]
14                                           [charter.net]
15                                  [verizon.net, cox.net]
16                                    [dtaa.com, dtaa.co

In [26]:
email_df['suspicious'] = email_df['to'].apply(lambda emails: any(
    re.match(r'.+@(.+)', e).group(1) != 'dtaa.com' for e in emails
))

In [41]:
email_df[email_df['from'].str.contains('derrick')]

,activity,attachments,bcc,cc,content,date,from,id,pc,size,to,user,suspicious,domain_from,domain_to


In [27]:
email_df['suspicious'] = email_df['suspicious'] & pd.isnull(email_df['bcc'])
email_df['suspicious'] = email_df['suspicious'] & pd.isnull(email_df['cc'])
email_df['suspicious'] = email_df['suspicious'] & pd.notnull(email_df['attachments'])
email_df['suspicious'] = email_df['suspicious'] & (email_df['to'].apply(len) == 1)
email_df['suspicious'] = email_df['suspicious'] & (email_df['activity'] == 'Send')

In [28]:
sdf = email_df[email_df['suspicious']]

In [33]:
sdf[sdf['from'] == 'Anjolie_Houston@optonline.net']

,activity,attachments,bcc,cc,content,date,from,id,pc,size,to,user,suspicious
1495,Send,allowing.pdf(641973);path.txt(1629722);strong....,NaN,NaN,using enlisted en 8 marriage rabaul strong ask...,2010-03-04 13:59:29,Anjolie_Houston@optonline.net,{N8Q1-B0WV70CW-4732CGTD},PC-8177,4169779,[Nichole_Conner@netzero.com],AEH0001,True
2035,Send,69.pdf(458655),NaN,NaN,general retreating five 8 north for parents 69...,2010-11-08 11:35:06,Anjolie_Houston@optonline.net,{P6Q3-L8QI75ZY-6943JHQL},PC-8177,475442,[Nichole_Conner@netzero.com],AEH0001,True
11817,Send,11.doc(731860),NaN,NaN,agency area verge philip august target king ca...,2010-05-04 08:41:49,Anjolie_Houston@optonline.net,{D6R2-J3SR37LE-3544OBRG},PC-8177,756275,[Nichole_Conner@netzero.com],AEH0001,True
14651,Send,suffered.doc(1981492),NaN,NaN,shaped generally anonymous another explanation...,2010-04-15 16:25:34,Anjolie_Houston@optonline.net,{B0E3-E6BB43FQ-8433MWQY},PC-8177,2012757,[Eden.T.Woodward@verizon.net],AEH0001,True
15552,Send,have.doc(2042804),NaN,NaN,flyby behind find certain bodies at least roug...,2010-05-27 14:05:19,Anjolie_Houston@optonline.net,{M7A5-P3IE92GC-7938VZFB},PC-8177,2082953,[Bruno_Winters@gmail.com],AEH0001,True
23099,Send,version.zip(507077),NaN,NaN,put married possible indian indicated until 19...,2010-06-24 12:41:18,Anjolie_Houston@optonline.net,{Z6F0-O5VI78MO-9687EIJE},PC-8177,530172,[Riggs_Jamalia@verizon.net],AEH0001,True


In [32]:
sdf.groupby('from').count().sort_values(by='activity')

,activity,attachments,bcc,cc,content,date,id,pc,size,to,user,suspicious
from,,,,,,,,,,,,
AFL2478@juno.com,1,1,0,0,1,1,1,1,1,1,1,1
Lindsay-William@juno.com,1,1,0,0,1,1,1,1,1,1,1,1
Logan-Sebastian@sbcglobal.net,1,1,0,0,1,1,1,1,1,1,1,1
Lucas_Vargas@earthlink.net,1,1,0,0,1,1,1,1,1,1,1,1
MGH883@juno.com,1,1,0,0,1,1,1,1,1,1,1,1
MJR54@aol.com,1,1,0,0,1,1,1,1,1,1,1,1
Madaline.N.Barnett@yahoo.com,1,1,0,0,1,1,1,1,1,1,1,1
Malachi.Jack.Riddle@dtaa.com,1,1,0,0,1,1,1,1,1,1,1,1
Mann_Fuller@yahoo.com,1,1,0,0,1,1,1,1,1,1,1,1


In [47]:
email_df.head()

,activity,attachments,bcc,cc,content,date,from,id,pc,size,to,user,suspicious,domain_from,domain_to
0,View,NaN,NaN,Omar.Kennedy.Perkins@dtaa.com,"There was also a tailor and a barber, and pris...",2010-07-07 10:45:27,Cecilia.Petra.Cantu@dtaa.com,{B8Y0-V4OS52TX-3095VRDC},PC-0251,33582,"[Katelyn.Lacota.Mclaughlin@dtaa.com, Prescott....",PES2571,False,dtaa.com,"[dtaa.com, dtaa.com]"
1,Send,C:\A25FAZKO.zip(561373),NaN,Armand.Wang.Savage@dtaa.com,In a letter written to Minister of Foreign Aff...,2010-05-25 12:07:42,Brennan.Barry.Joseph@dtaa.com,{Q1M1-I0JN15JG-9093BGJZ},PC-2745,595800,[Ciaran.Nicolas.Sherman@dtaa.com],BBJ1992,False,dtaa.com,[dtaa.com]
2,View,NaN,NaN,NaN,"To slow the destruction by sulfuric acid, diff...",2010-05-17 12:57:02,Rajah.Xander.Lindsey@dtaa.com,{D8S1-Y6VE12JH-3081NHMR},PC-2965,27400,[Keelie.Ramona.Haynes@dtaa.com],KRH1258,False,dtaa.com,[dtaa.com]
3,Receive,NaN,NaN,NaN,assigning communication replied fe defence ele...,2010-03-19 11:34:36,Hayes.Tobias.Hendricks@dtaa.com,{F1J6-F1NZ76CK-9902SOEC},PC-3335,31511,[Rae.Wilma.Roman@dtaa.com],RWR0991,False,dtaa.com,[dtaa.com]
4,Send,NaN,Imelda.Macey.Kline@dtaa.com,Kameko.Zenaida.Mccullough@dtaa.com;Jane.Leila....,"The increase in speed, armour and gun size for...",2010-05-20 10:30:53,Imelda.Macey.Kline@dtaa.com,{S1V8-V7OY62PP-8116HWVN},PC-6745,24264,[Tobias.Stephen.Contreras@dtaa.com],IMK3365,False,dtaa.com,[dtaa.com]


Could do extra cols like:
- number of attachements
- BCC present or not
- CC present or not
- number of CC/BCC/TO
- content length
- external domain present or not? (cc, bcc, to)

Need capture however also interactions between rows
- same person sending several times etc...
- could groupby person?
- number of email, of unique recipients, CC, BCC, ... 
- PCs sent from
- size of attachements...


In [65]:
device_df.head()

,activity,date,file_tree,id,pc,user,ft_count
0,Connect,2010-06-01 18:10:35,R:\;R:\MAL1912,{M4H6-P5FR33RV-6020CMPV},PC-3117,MAL1912,2
1,Disconnect,2010-07-30 11:19:40,NaN,{K8G4-A7FO12TE-2893OGZJ},PC-7078,SIH2331,0
2,Connect,2010-03-29 13:26:43,R:\;R:\13x1p75;R:\27QJ5H2;R:\73ln1M2;R:\ATH0225,{Y2E7-P4UT29OH-0974BEHD},PC-7844,ATH0225,5
3,Disconnect,2011-01-21 11:00:53,NaN,{C3J4-B7YG07GJ-5906YIFV},PC-0176,OLR0348,0
4,Connect,2010-03-02 17:05:56,R:\;R:\75Q85F6;R:\ZED2896,{U0P5-A7SK79FD-2677GWJO},PC-5930,ZED2896,3
